db에서 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine

In [2]:
# 1. 데이터베이스 연결 -> 연결 객체 반환
conn = pymysql.connect(host="localhost", port=3306, db="mp",
                       user="root", passwd="1234", cursorclass=pymysql.cursors.DictCursor)

cursor = conn.cursor()

In [3]:
# 3. SQL 작성 + 명령 실행 ( 조회 명령 )
sql = """select * from fine_dust"""
cursor.execute(sql)
result = cursor.fetchall()
result

[{'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-01',
  'fine_dust(㎍/㎥)': 36},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-02',
  'fine_dust(㎍/㎥)': 43},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-03',
  'fine_dust(㎍/㎥)': 47},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-04',
  'fine_dust(㎍/㎥)': 50},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-05',
  'fine_dust(㎍/㎥)': 37},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-06',
  'fine_dust(㎍/㎥)': 30},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-07',
  'fine_dust(㎍/㎥)': 56},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-08',
  'fine_dust(㎍/㎥)': 27},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-09',
  'fine_dust(㎍/㎥)': 33},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021-01-10',
  'fine_dust(㎍/㎥)': 48},
 {'station_id': 108,
  'station_name': '서울',
  'date': '2021

In [4]:
fine_dust_df = pd.DataFrame(result)
fine_dust_df.head()

,station_id,station_name,date,fine_dust(㎍/㎥)
0,108,서울,2021-01-01,36
1,108,서울,2021-01-02,43
2,108,서울,2021-01-03,47
3,108,서울,2021-01-04,50
4,108,서울,2021-01-05,37


In [5]:
conn.close()
cursor.close()

In [6]:
fine_dust_df.to_csv('./data/fine_dust.csv')

In [ ]:
# conn = pymysql.connect(host="localhost", port=3306, db="project", 
#                        user="humanda", password="humanda")


# sql3 = "select * from transportation"

# transportation = pd.read_sql_query(sql3, conn)

### db 종료 후 DF으로 작업하기

In [24]:
fine_dust_df

,station_id,station_name,date,fine_dust(㎍/㎥)
0,108,서울,2021-01-01,36
1,108,서울,2021-01-02,43
2,108,서울,2021-01-03,47
3,108,서울,2021-01-04,50
4,108,서울,2021-01-05,37
...,...,...,...,...
1318,108,서울,2024-10-25,29
1319,108,서울,2024-10-26,26
1320,108,서울,2024-10-27,27
1321,108,서울,2024-10-28,38
